In [51]:
import sqlite3
import pandas as pd

con = sqlite3.connect('sql-murder-mystery.db')
pd.set_option('display.max_colwidth', None)

In [52]:
q = """
SELECT name 
FROM sqlite_master
WHERE type = 'table'
"""

pd.read_sql(q, con)

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In [53]:
q = """
SELECT sql
FROM sqlite_master
WHERE name = 'crime_scene_report'
"""

pd.read_sql(q,con)


,sql
0,"CREATE TABLE crime_scene_report (\n date integer,\n type text,\n description text,\n city text\n )"


In [54]:
q = """
SELECT *
FROM crime_scene_report
WHERE city = 'SQL City' AND date = '20180115' AND type = 'murder'
"""

pd.read_sql(q,con)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [ ]:
# q = """

# WITH aux as (
# Select *
# FROM person
# WHERE address_street_name = 'Northwestern Dr'
# ORDER BY address_number DESC
# LIMIT 1
# ),

# aux2 as (
# SELECT *
# FROM person 
# WHERE address_street_name = "Franklin Ave" and name LIKE 'Annabel%'
# )

# SELECT *
# FROM aux 
# JOIN aux2 ON aux.name = aux2.name


# """

# pd.read_sql(q,con)

,id,name,license_id,address_number,address_street_name,ssn,id,name,license_id,address_number,address_street_name,ssn


In [55]:
q = """
SELECT *
FROM
(
Select *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1
)
UNION ALL
SELECT *
FROM person 
WHERE address_street_name = "Franklin Ave" and name LIKE 'Annabel%'
"""

pd.read_sql(q,con)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [56]:
q = """
SELECT *
FROM interview 
"""

pd.read_sql(q,con)

,person_id,transcript
0,28508,‘I deny it!’ said the March Hare.\n
1,63713,\n
2,86208,"way, and the whole party swam to the shore.\n"
3,35267,"lessons in here? Why, there’s hardly room for YOU, and no room at all\n"
4,33856,\n
...,...,...
4986,37357,"Alice did not wish to offend the Dormouse again, so she began very\n"
4987,10206,"time,’ she said, ‘than waste it in asking riddles that have no answers.’\n"
4988,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
4989,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [58]:
q = """
SELECT person.*, interview.*
FROM
(
Select *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1
) AS nw_person
JOIN interview ON nw_person.id = interview.person_id

UNION ALL

SELECT person.*, interview.*
FROM person 
JOIN interview ON person.id = interview.person_id
WHERE address_street_name = "Franklin Ave" and name LIKE 'Annabel%'

"""

pd.read_sql(q,con)

DatabaseError: Execution failed on sql '
SELECT person.*, interview.*
FROM
(
Select *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1
) AS nw_person
JOIN interview ON nw_person.id = interview.person_id

UNION ALL

SELECT person.*, interview.*
FROM person 
JOIN interview ON person.id = interview.person_id
WHERE address_street_name = "Franklin Ave" and name LIKE 'Annabel%'

': no such table: person

In [ ]:
q = """
-- First SELECT: person from Northwestern Dr with highest address
SELECT p1.*, i1.*
FROM (
    SELECT *
    FROM person
    WHERE address_street_name = 'Northwestern Dr'
    ORDER BY address_number DESC
    LIMIT 1
) AS p1
JOIN interview AS i1 ON p1.id = i1.person_id

UNION ALL

-- Second SELECT: person on Franklin Ave named Annabel...
SELECT p2.*, i2.*
FROM person AS p2
JOIN interview AS i2 ON p2.id = i2.person_id
WHERE p2.address_street_name = 'Franklin Ave'
  AND p2.name LIKE 'Annabel%';
"""

pd.read_sql(q, con)

,id,name,license_id,address_number,address_street_name,ssn,person_id,transcript
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,Annabel Miller,490173,103,Franklin Ave,318771143,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [64]:
q = """
SELECT * 
FROM get_fit_now_member
WHERE id LIKE '48Z%' AND membership_status = 'gold'
"""

pd.read_sql(q,con)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [71]:
q = """
SELECt * 
FROM drivers_license
WHERE plate_number LIKE '%H42W%' AND gender = 'male'
"""
pd.read_sql(q,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [70]:
q = """

WITH members AS (
SELECT * 
FROM get_fit_now_check_in
WHERE check_in_date = '20180109' AND membership_id LIKE '48Z%'
)
SELECT *
FROM get_fit_now_member
JOIN members ON get_fit_now_member.id = members.membership_id
"""
pd.read_sql(q,con)

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,28819,Joe Germuska,20160305,gold,48Z7A,20180109,1600,1730
1,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700


In [73]:
q = """

SELECT * 
FROM person 
WHERE name  = 'Joe Germuska' OR name = 'Jeremy Bowers'
"""
pd.read_sql(q,con)

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [74]:
q = """
-- First SELECT: Joe Germuska
SELECT p1.*, i1.*
FROM (
    SELECT *
    FROM person
    WHERE id = '28819'
) AS p1
JOIN interview AS i1 ON p1.id = i1.person_id

UNION ALL

-- Second SELECT: Jeremy Bowers
SELECT p2.*, i2.*
FROM person AS p2
JOIN interview AS i2 ON p2.id = i2.person_id
WHERE id = '67318'

"""

pd.read_sql(q, con)

,id,name,license_id,address_number,address_street_name,ssn,person_id,transcript
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [84]:
q = """
SELECT *
FROM drivers_license
WHERE car_make = 'Tesla' AND car_model = 'Model S' AND gender = 'female' AND hair_color = 'red'
"""

pd.read_sql(q, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [106]:
q = """
SELECT *
FROM facebook_event_checkin
WHERE event_name = 'SQL Symphony Concert' AND date LIKE '201712%'
GROUP BY person_id
HAVING COUNT(*) = 3
ORDER BY person_id

"""

pd.read_sql(q, con)

,person_id,event_id,event_name,date
0,24556,1143,SQL Symphony Concert,20171207
1,99716,1143,SQL Symphony Concert,20171206


In [125]:
q = """
WITH event_checkins AS (
SELECT *
FROM facebook_event_checkin
WHERE event_name = 'SQL Symphony Concert' AND date LIKE '201712%'
GROUP BY person_id
HAVING COUNT(*) = 3
ORDER BY person_id
),

suspects AS (
SELECT id, name, license_id, ssn
FROM person 
JOIN event_checkins ON person.id = event_checkins.person_id
)

SELECT *
FROM suspects
JOIN income ON suspects.ssn = income.ssn
"""

pd.read_sql(q, con)

,id,name,license_id,ssn,ssn,annual_income
0,99716,Miranda Priestly,202298,987756388,987756388,310000


In [126]:
q = """
SELECT *
FROM solution

"""

pd.read_sql(q, con)

,user,value
